# Make Lines
In this notebook we take the NARN database and take the lines from it to place into the database. Many of these lines are not used in the routes but are included for completeness.

The process starts with determining if the Line has rights for any of the major railroads. If it does, then the Line is added to the database. At a later time, these lines can be used to determine routes. If the line is part of a route it can then be used to get associated elevations.

The latest version of the NARN database uses MultiLineStrings now instead of LineStrings. This data is converted to arrays of data instead and is saved in the database. Additionally, this data is modified to better spread and cover the line represented by the locations.

In [1]:
import geopandas as gpd
import networkx as nx
from shapely import geometry, ops
import numpy as np
import matplotlib.pyplot as plt
import requests
import pandas as pd
import folium
import time
from requests.auth import AuthBase
import os
from scipy.spatial import distance

In [2]:
data = gpd.read_file("North_American_Rail_Network_Lines.geojson")

ERROR 1: PROJ: proj_create_from_database: Open of /opt/conda/share/proj failed


The column used to all be capitalized, now they are lower case.

In [3]:
data.columns

Index(['objectid', 'fraarcid', 'frfranode', 'tofranode', 'stfips', 'cntyfips',
       'stcntyfips', 'stateab', 'country', 'fradistrct', 'rrowner1',
       'rrowner2', 'rrowner3', 'trkrghts1', 'trkrghts2', 'trkrghts3',
       'trkrghts4', 'trkrghts5', 'trkrghts6', 'trkrghts7', 'trkrghts8',
       'trkrghts9', 'division', 'subdiv', 'branch', 'yardname', 'passngr',
       'stracnet', 'tracks', 'net', 'miles', 'km', 'timezone', 'shape_Length',
       'geometry'],
      dtype='object')

The geometry element in the data consists of MultiLineString for some reason. We need to convert this to LineStrings? 

In [4]:
data6 = data.explode(index_parts=True)

In [5]:
data5 = data6.to_crs(3857)

We can simplify the linestaings, but the result of this is just geometry - we lose the dataframe.

In [6]:
data5.geometry = data5.simplify(1, preserve_topology=False)

We also need to add a locations to reduce maximum distance between locations.

In [7]:
data5.geometry = data5.segmentize(max_segment_length=200)

In [8]:
data2 = data5.to_crs(4326)

In [9]:
URL ="http://score-web-1:8000"
URL1 = URL+"/api-token-auth/"
payload = {'username':'locomotives', 'password':'locomotives'}
URL2 = URL+"/api/line/add/"
URL3 = URL+'/api/railroad/'
URL4 = URL + "/api/line/"

In [10]:
t = requests.post(URL1, data=payload )
token = t.json().get('token')

In [11]:
class TokenAuth(AuthBase):
    """ Implements a custom authentication scheme. """

    def __init__(self, token):
        self.token = token

    def __call__(self, r):
        """ Attach an API token to a custom auth header. """
        r.headers['Authorization'] = "Token " + f'{self.token}'
        return r

In [12]:
r = requests.get(URL3, auth=TokenAuth(token))
railroads=r.json()['results']
railroads

[{'id': 1, 'code': 'BNSF', 'name': 'Burlington Northern and Santa Fe'},
 {'id': 2, 'code': 'CN', 'name': 'Canadian National Railway'},
 {'id': 3, 'code': 'CP', 'name': 'Canadian Pacific Railway'},
 {'id': 4, 'code': 'CSXT', 'name': 'CSX Transportation'},
 {'id': 5, 'code': 'NS', 'name': 'Norfolk Southern Railway'},
 {'id': 6, 'code': 'KCS', 'name': 'Kansas City Southern Railway'},
 {'id': 7, 'code': 'UP', 'name': 'Union Pacific'}]

In [13]:
def curve(p1, p2, p3):
    v1=p2-p1
    v2=p3-p2
    v3=p1-p3
    d1=np.linalg.norm(v1)
    d2=np.linalg.norm(v2)
    d3=np.linalg.norm(v3)
    num = 1746.375*2*np.linalg.norm(np.cross(v1,v2))
    d = num/(d1*d2*d3)
    if d<0.01:
        d=0.0
    return d

The process will go through each line segment that is in the NARN database and determine which of the 7 Class 1 railroads may have rights. If any have rigfhts it is included in the database and further processing is completed.

The next step is to simplify and segmentize the data.

We can include curvature at this point also, but not go through the expense of determining elevations and gradients.
This will be delayed until we cna determine if the Line segment is part of a route of interest.

In [14]:
codes = [d['code'] for d in railroads]
codes

['BNSF', 'CN', 'CP', 'CSXT', 'NS', 'KCS', 'UP']

In [15]:
ids = [d['id'] for d in railroads]
ids

[1, 2, 3, 4, 5, 6, 7]

Need to do some work on converting lat longs and x, y in meters.

In [16]:
data2_xy = data2.to_crs(3857)

Still need to simplify and segmentize the xy data.

In [17]:
for index, row in data2.iterrows():
    # if index[0] < 10:
    if True:
        if row[['rrowner1', 'rrowner2', 'rrowner3', 'trkrghts1', 'trkrghts2', 'trkrghts3', 'trkrghts4', 'trkrghts5', 'trkrghts6', 'trkrghts7', 'trkrghts8', 'trkrghts9']].isin(codes).any():
            rights = []
            for id, code in zip(ids, codes):
                if row[['rrowner1', 'rrowner2', 'rrowner3', 'trkrghts1', 'trkrghts2', 'trkrghts3', 'trkrghts4', 'trkrghts5', 'trkrghts6', 'trkrghts7', 'trkrghts8', 'trkrghts9']].isin([code]).any():
                    rights.append(code)
            rowxy = data2_xy.iloc[index[0]]
            p = np.array(rowxy.geometry.coords)
            xy = p[:,0:2]
            # we want the interpoint distance between the points - the offset of 1, diagonal of the cdist matrix
            dist = np.diagonal(distance.cdist(xy, xy), offset=1)
            # lets not get elevations for now
            lnglat = np.array(row.geometry.coords)
            curvature=[]
            if (len(xy)>2):
                for i in range(len(xy)-2):
                    curvature.append(curve(xy[i],xy[i+1],xy[i+2]))
                curvature.append(curvature[-1])
            else:
                curvature = [0.0]
            line = {
                "fra_id" : rowxy['fraarcid'],
                "from_node" : rowxy['frfranode'],
                "to_node" : rowxy['tofranode'],
                "length" : dist.sum(),
                "net": rowxy['net'],
                "rights" : rights,
                "xy": xy.tolist(),
                "lnglat": lnglat.tolist(),
                "curvature": curvature,
                "distance":dist.tolist()
            }
            requests.post(URL2, data=line, auth=TokenAuth(token))            
    